In [1]:
# cell 02
import sagemaker
bucket=sagemaker.Session().default_bucket()
prefix = 'sagemaker/simple-xgboost-paysim'
 
# Define IAM role
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import os

In [5]:
data = pd.read_csv('./dataset/raw/raw_paysim.csv')
pd.set_option('display.max_rows', 20)         # Keep the output on one page
data

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.00,0.00,0.0,0.0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.00,0.00,0.0,0.0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.00,0.00,1.0,0.0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.00,0.00,1.0,0.0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.00,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1577096,155,TRANSFER,43450.49,C730444513,0.0,0.00,C814720871,165948.65,1270011.14,0.0,0.0
1577097,155,TRANSFER,49185.39,C1893451872,0.0,0.00,C1661673055,1899236.38,1733474.42,0.0,0.0
1577098,155,TRANSFER,120655.98,C746221708,0.0,0.00,C1786396642,6672282.71,6792938.70,0.0,0.0
1577099,155,TRANSFER,755659.76,C1834005599,0.0,0.00,C1985986684,1509822.54,2265482.30,0.0,0.0


In [6]:
#drop last row
data.dropna()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.00,0.00,0.0,0.0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.00,0.00,0.0,0.0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.00,0.00,1.0,0.0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.00,0.00,1.0,0.0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.00,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1577095,155,TRANSFER,214977.94,C624724710,0.0,0.00,C1294409003,831919.08,1046897.02,0.0,0.0
1577096,155,TRANSFER,43450.49,C730444513,0.0,0.00,C814720871,165948.65,1270011.14,0.0,0.0
1577097,155,TRANSFER,49185.39,C1893451872,0.0,0.00,C1661673055,1899236.38,1733474.42,0.0,0.0
1577098,155,TRANSFER,120655.98,C746221708,0.0,0.00,C1786396642,6672282.71,6792938.70,0.0,0.0


In [7]:
# one hot encoding for type column
one_hot = pd.get_dummies(data['type'])
data = data.drop('type',axis = 1)
data = data.join(one_hot)
data

,step,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,CASH_IN,CASH_OUT,DEBIT,PAYMENT,TRANSFER
0,1,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.00,0.00,0.0,0.0,0,0,0,1,0
1,1,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.00,0.00,0.0,0.0,0,0,0,1,0
2,1,181.00,C1305486145,181.0,0.00,C553264065,0.00,0.00,1.0,0.0,0,0,0,0,1
3,1,181.00,C840083671,181.0,0.00,C38997010,21182.00,0.00,1.0,0.0,0,1,0,0,0
4,1,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.00,0.00,0.0,0.0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1577096,155,43450.49,C730444513,0.0,0.00,C814720871,165948.65,1270011.14,0.0,0.0,0,0,0,0,1
1577097,155,49185.39,C1893451872,0.0,0.00,C1661673055,1899236.38,1733474.42,0.0,0.0,0,0,0,0,1
1577098,155,120655.98,C746221708,0.0,0.00,C1786396642,6672282.71,6792938.70,0.0,0.0,0,0,0,0,1
1577099,155,755659.76,C1834005599,0.0,0.00,C1985986684,1509822.54,2265482.30,0.0,0.0,0,0,0,0,1


In [8]:
#le = LabelEncoder()
#encoded_nameorig = le.fit_transform(data['nameOrig'])
#encoded_namedest = le.fit_transform(data['nameDest'])
#data['nameOrig'] = encoded_nameorig.tolist()
#data['nameDest'] = encoded_namedest.tolist()
data = data.drop('nameOrig', axis=1)
data = data.drop('nameDest', axis=1)
data

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,CASH_IN,CASH_OUT,DEBIT,PAYMENT,TRANSFER
0,1,9839.64,170136.0,160296.36,0.00,0.00,0.0,0.0,0,0,0,1,0
1,1,1864.28,21249.0,19384.72,0.00,0.00,0.0,0.0,0,0,0,1,0
2,1,181.00,181.0,0.00,0.00,0.00,1.0,0.0,0,0,0,0,1
3,1,181.00,181.0,0.00,21182.00,0.00,1.0,0.0,0,1,0,0,0
4,1,11668.14,41554.0,29885.86,0.00,0.00,0.0,0.0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1577096,155,43450.49,0.0,0.00,165948.65,1270011.14,0.0,0.0,0,0,0,0,1
1577097,155,49185.39,0.0,0.00,1899236.38,1733474.42,0.0,0.0,0,0,0,0,1
1577098,155,120655.98,0.0,0.00,6672282.71,6792938.70,0.0,0.0,0,0,0,0,1
1577099,155,755659.76,0.0,0.00,1509822.54,2265482.30,0.0,0.0,0,0,0,0,1


In [9]:
train_data, test_data = np.split(data, [int(0.8 * len(data))])

In [10]:
pd.concat([train_data['isFraud'], train_data.drop(['isFraud'], axis=1)], axis=1).to_csv('/home/ec2-user/SageMaker/xgboost_train.csv', index=False, header=False)

In [11]:
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/xgboost_train.csv')).upload_file('xgboost_train.csv')

In [12]:
container = sagemaker.image_uris.retrieve(region=boto3.Session().region_name, framework='xgboost', version='latest')

In [13]:
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')

In [14]:
sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    instance_count=1, 
                                    instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        num_round=100)

xgb.fit({'train': s3_input_train}) 

2022-09-09 09:28:22 Starting - Starting the training job...ProfilerReport-1662715702: InProgress
...
2022-09-09 09:29:06 Starting - Preparing the instances for training.........
2022-09-09 09:30:47 Downloading - Downloading input data......
2022-09-09 09:31:47 Training - Downloading the training image.....Arguments: train
[2022-09-09:09:32:30:INFO] Running standalone xgboost training.
[2022-09-09:09:32:30:INFO] Path /opt/ml/input/data/validation does not exist!
[2022-09-09:09:32:30:INFO] File size need to be processed in the node: 71.5mb. Available memory size in the node: 8793.83mb
[2022-09-09:09:32:30:INFO] Determined delimiter of CSV input is ','
[09:32:30] S3DistributionType set as FullyReplicated
[09:32:31] 1261680x12 matrix with 15140160 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[09:32:32] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 0 pruned nodes, max_depth=5
[0]#011train-error:0.00089
[09:32:33] src/tree/upda

[09:33:15] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 8 pruned nodes, max_depth=5
[47]#011train-error:0.000217
[09:33:16] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 6 pruned nodes, max_depth=5
[48]#011train-error:0.000212
[09:33:17] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 2 pruned nodes, max_depth=5
[49]#011train-error:0.000211
[09:33:18] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 2 pruned nodes, max_depth=5
[50]#011train-error:0.000207
[09:33:18] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 2 pruned nodes, max_depth=5
[51]#011train-error:0.000203
[09:33:19] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 10 pruned nodes, max_depth=5
[52]#011train-error:0.000202
[09:33:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 2 pruned nodes, max_depth=5
[53]#011train-error:0.000201
[09:33:21] s


2022-09-09 09:34:28 Uploading - Uploading generated training model
2022-09-09 09:34:28 Completed - Training job completed
ProfilerReport-1662715702: NoIssuesFound
Training seconds: 223
Billable seconds: 223


In [15]:
xgb_predictor = xgb.deploy(initial_instance_count=1,
                           instance_type='ml.m4.xlarge')

------!

In [16]:
xgb_predictor.serializer = sagemaker.serializers.CSVSerializer()

In [17]:
def predict(data, predictor, rows=500 ):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

predictions = predict(test_data.drop(['isFraud'], axis=1).to_numpy(), xgb_predictor)

In [18]:
# cell 21
pd.crosstab(index=test_data['isFraud'], columns=np.round(predictions), rownames=['actuals'], colnames=['predictions'])

predictions,0.0,1.0
actuals,,
0.0,315196,0
1.0,74,150


In [29]:
# cell 22
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
hyperparameter_ranges = {'eta': ContinuousParameter(0, 1),
                            'min_child_weight': ContinuousParameter(1, 10),
                            'alpha': ContinuousParameter(0, 2),
                            'max_depth': IntegerParameter(1, 10)}

In [30]:
objective_metric_name = 'validation:auc'
tuner = HyperparameterTuner(xgb,
                            objective_metric_name,
                            hyperparameter_ranges,
                            max_jobs=20,
                            max_parallel_jobs=3)

In [ ]:
tuner.fit({'train': s3_input_train})

In [ ]:
boto3.client('sagemaker').describe_hyper_parameter_tuning_job(
HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)['HyperParameterTuningJobStatus']

In [ ]:
# cell 27
# return the best training job name
tuner.best_training_job()

In [ ]:
tuner_predictor = tuner.deploy(initial_instance_count=1,
                           instance_type='ml.m4.xlarge')

In [ ]:
tuner_predictor.serializer = sagemaker.serializers.CSVSerializer()

In [ ]:
predictions = predict(test_data.drop(['isFraud'], axis=1).to_numpy(), tuner_predictor)

In [ ]:
pd.crosstab(index=test_data['isFraud'], columns=np.round(predictions), rownames=['actuals'], colnames=['predictions'])